In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2

In [7]:
images_yes = [cv2.imread(file) for file in glob.glob("data/yes/*.*")]
images_no = [cv2.imread(file) for file in glob.glob("data/no/*.*")]
len(images_no)

98

In [8]:
def image_crop(image_path):
    cropped_image = []
    for i in image_path:
        if i is None:
            pass
        else:
            image = cv2.resize(i , (128,128))
        
        #The initial processing of the image
        image = cv2.medianBlur(image, 3)
        image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #The declaration of CLAHE 
        #clipLimit -> Threshold for contrast limiting
        clahe = cv2.createCLAHE(clipLimit = 5)
        final_img = clahe.apply(image_bw)
        cropped_image.append(final_img)
    cropped_image = np.array(cropped_image)
    return cropped_image

In [9]:
cropped_images_no = image_crop(images_no)
cropped_images_yes = image_crop(images_yes)

In [10]:
y_yes = np.ones(len(cropped_images_yes), dtype="int8")
y_no = np.zeros(len(cropped_images_no), dtype="int8")

In [12]:
X = np.concatenate((cropped_images_yes, cropped_images_no), axis=0)
y = np.concatenate((y_yes, y_no), axis=0)

d1, d2, d3 = X.shape
X = X.reshape((d1, d2 * d3))

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

### PCA

In [15]:
from sklearn.decomposition import PCA, KernelPCA

pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_sc)
X_test_pca = pca.transform(X_test_sc)

In [16]:
kpca = KernelPCA()
X_train_kpca = kpca.fit_transform(X_train_sc)
X_test_kpca = kpca.transform(X_test_sc)

### SVC with PCA

In [17]:
from sklearn.svm import SVC

svc_pca = SVC(kernel="linear")
svc_pca.fit(X_train_pca, y_train)
y_predict_pca = svc_pca.predict(X_test_pca)

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predict_pca))

              precision    recall  f1-score   support

           0       0.79      0.79      0.79        24
           1       0.81      0.81      0.81        27

    accuracy                           0.80        51
   macro avg       0.80      0.80      0.80        51
weighted avg       0.80      0.80      0.80        51



### SVC with kernel PCA

In [19]:
svc_kpca = SVC(kernel="linear")
svc_kpca.fit(X_train_kpca, y_train)
y_predict_kpca = svc_kpca.predict(X_test_kpca)

In [21]:
print(classification_report(y_test, y_predict_kpca))

              precision    recall  f1-score   support

           0       0.85      0.71      0.77        24
           1       0.77      0.89      0.83        27

    accuracy                           0.80        51
   macro avg       0.81      0.80      0.80        51
weighted avg       0.81      0.80      0.80        51



### SVC without PCA

In [18]:
svc = SVC(kernel="linear")

svc.fit(X_train_sc, y_train)
y_predict = svc.predict(X_test_sc)

In [22]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.85      0.71      0.77        24
           1       0.77      0.89      0.83        27

    accuracy                           0.80        51
   macro avg       0.81      0.80      0.80        51
weighted avg       0.81      0.80      0.80        51

